In [ ]:
import copy
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import inspect
import pandas as pd

sys.path.insert(0, "..")
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
Q1_THEORY_DIR = "../dump/q1_theory/"
Q1_BOXATK_DIR = "../dump/q1_boxatk/"
Q2_DIR = "../dump/q2"
Q3_DIR = "../dump/q3_sparsity"

In [ ]:
#######################################################
# Q1 ##################################################
#######################################################

In [ ]:
def load_q1_theory(model_type, method_type, top_frac, lambd):
    model_type = model_type.lower()
    method_type = method_type.lower()
    method_type = "vgradu" if method_type == "vgrad" else method_type
    method_type = "igradu" if method_type == "igrad" else method_type
    
    if model_type == "roberta":
        csv_file = f"q1t_{model_type}_q64_{method_type}_top{top_frac:.4f}_lam{lambd:.4f}.csv"
    else:
        csv_file = f"q1t_{model_type}_psz28_q64_{method_type}_top{top_frac:.4f}_lam{lambd:.4f}.csv"
    csv_file = os.path.join(Q1_THEORY_DIR, csv_file)
    return pd.read_csv(csv_file)

def load_q1_boxatk(model_type, method_type, top_frac, lambd):
    if model_type == "roberta":
        csv_file = f"q1e_{model_type}_q16_{method_type}_top{top_frac:.4f}_lam{lambd:.4f}.csv"
    else:
        csv_file = f"q1e_{model_type}_psz28_q16_{method_type}_top{top_frac:.4f}_lam{lambd:.4f}.csv"
    csv_file = os.path.join(Q1_BOXATK_DIR, csv_file)
    return pd.read_csv(csv_file)

# Certified stuff
def q1t_drops(df, rs, prop="inc", consistent=True, div_by_p=False, use_mu=False):
    assert prop in ["inc", "dec"]
    
    true_labels = df["true_label"].to_numpy()
    ones_labels = (df["ones_mu_label"] if use_mu else df["ones_label"]).to_numpy()
    exbs_labels = (df["exbs_mu_label"] if use_mu else df["exbs_label"]).to_numpy()
    ones_gaps = (df["ones_mu_gap"] if use_mu else df["ones_gap"]).to_numpy()
    exbs_gaps = (df["exbs_mu_gap"] if use_mu else df["exbs_gap"]).to_numpy()
    todo_labels = exbs_labels if prop == "inc" else ones_labels
    todo_gaps = exbs_gaps if prop == "inc" else ones_gaps
    
    N = len(df)
    lambd = df["lambd"][0]
    cert_rs = todo_gaps / (2 * lambd)
    
    if div_by_p:
        ps = df["p"].to_numpy()
        cert_rs = cert_rs / ps
    
    if consistent:
        hit_bits = ones_labels == exbs_labels
#         hit_bits = true_labels == exbs_labels
#         hit_bits = true_labels == true_labels
#         hit_bits = true_labels == todo_labels
#         print(f"num hits {hit_bits.sum()}")
        drops = np.array([hit_bits[cert_rs >= r].sum() / N for r in rs])
    else:
        drops = np.array([(cert_rs >= r).sum() / N for r in rs])

    return drops

# Box attack stuff
def q1e_drops(df, rs, prop="inc", consistent=True, div_by_p=False, use_mu=False):
    assert prop in ["inc", "dec"]
    
    true_labels = df["true_label"].to_numpy()
    ones_labels = (df["ones_mu_label"] if use_mu else df["ones_label"]).to_numpy()
    exbs_labels = (df["exbs_mu_label"] if use_mu else df["exbs_label"]).to_numpy()
    todo_labels = exbs_labels if prop == "inc" else ones_labels
    r_maxs = (df["inc_curr_r_max"] if prop == "inc" else df["dec_curr_r_max"]).to_numpy()
        
    N = len(df)
    r_maxs = r_maxs / df["p"].to_numpy() if div_by_p else r_maxs
    if consistent:
        hit_bits = ones_labels == exbs_labels
        drops = np.array([hit_bits[r_maxs >= r].sum() / N for r in rs])
    else:
        drops = np.array([(r_maxs >= r).sum() / N for r in rs])
    return drops


In [ ]:
##################################################################
### Certified and empirical stability, only for SHAP top-25%
##################################################################

In [ ]:
def q1_plot_cert_vs_emp(model_type, trs, plot_trs, ers, plot_ers,
                        title = None,
                        lfs = 10,
                        afs = 12,
                        tfs = 14,
                        show_lhs = True,
                        subplot_titles = False):
    plt.clf()
    fig, ax = plt.subplots(2,2, figsize=(6.4 * 2, 4.8*1.2))

    q1t_lambds = [8/8, 4/8, 2/8, 1/8]
    q1e_lambds = [8/8, 4/8, 2/8, 1/8]
    q1t_dfs = [(l, load_q1_theory(model_type, "shap", 0.25, l)) for l in q1t_lambds]
    q1e_dfs = [(l, load_q1_boxatk(model_type, "shap", 0.25, l)) for l in q1e_lambds]
    div_by_p = model_type.lower() == "roberta"
    
    for (lambd, df) in q1t_dfs:
        inc_drops = q1t_drops(df, trs, prop="inc", div_by_p=div_by_p)
        dec_drops = q1t_drops(df, trs, prop="dec", div_by_p=div_by_p)
        if show_lhs:
            ax[0,0].plot(plot_trs, inc_drops, label=f"λ={int(lambd*8)}/8 ({inc_drops[0]:.3f})")
            ax[0,1].plot(plot_trs, dec_drops, label=f"λ={int(lambd*8)}/8 ({dec_drops[0]:.3f})")
        else:
            ax[0,0].plot(plot_trs, inc_drops, label=f"λ={int(lambd*8)}/8")
            ax[0,1].plot(plot_trs, dec_drops, label=f"λ={int(lambd*8)}/8")
            

    for (lambd, df) in q1e_dfs:
        inc_drops = q1e_drops(df, ers, prop="inc", div_by_p=div_by_p)
        dec_drops = q1e_drops(df, ers, prop="dec", div_by_p=div_by_p)
        if show_lhs:
            ax[1,0].plot(plot_ers, inc_drops, label=f"λ={int(lambd*8)}/8 ({inc_drops[0]:.3f})")
            ax[1,1].plot(plot_ers, dec_drops, label=f"λ={int(lambd*8)}/8 ({dec_drops[0]:.3f})")
        else:
            ax[1,0].plot(plot_ers, inc_drops, label=f"λ={int(lambd*8)}/8")
            ax[1,1].plot(plot_ers, dec_drops, label=f"λ={int(lambd*8)}/8")
        
    if subplot_titles:
        emp_legend_loc = "lower left" if model_type == "roberta" else "upper right"
        ax[0,0].legend(loc="upper right", fontsize=lfs, title_fontsize=lfs)
        ax[0,1].legend(loc="upper right", title_fontsize=lfs)
        ax[1,0].legend(loc=emp_legend_loc, fontsize=lfs, title_fontsize=lfs)
        ax[1,1].legend(loc=emp_legend_loc, fontsize=lfs, title_fontsize=lfs)
        
        ax[0,0].set_title("Certified Consistent and Inc Stable", fontsize=tfs)
        ax[0,1].set_title("Certified Consistent and Dec Stable", fontsize=tfs)
        ax[1,0].set_title("Empirical Consistent and Inc Stable", fontsize=tfs)
        ax[1,1].set_title("Empirical Consistent and Dec Stable", fontsize=tfs)
        fig.subplots_adjust(hspace=0.35)
        
    else:
        emp_legend_loc = "lower left" if model_type == "roberta" else "upper right"
        ax[0,0].legend(loc="upper right", title="Cert Cons+IncStab", fontsize=lfs, title_fontsize=lfs)
        ax[0,1].legend(loc="upper right", title="Cert Cons+DecStab", fontsize=lfs, title_fontsize=lfs)
        ax[1,0].legend(loc=emp_legend_loc, title="Emp Cons+IncStab", fontsize=lfs, title_fontsize=lfs)
        ax[1,1].legend(loc=emp_legend_loc, title="Emp Cons+DecStab", fontsize=lfs, title_fontsize=lfs)
        
    for i in [0,1]:
        for j in [0,1]:
            ax[i,j].set_yticks([0.0, 0.2, 0.4, 0.6, 0.8])
            ax[i,j].tick_params(axis="x", labelsize=lfs)
            ax[i,j].tick_params(axis="y", labelsize=lfs)

    ax[1,0].set_xlabel("Fraction of Features", fontsize=afs)
    ax[1,1].set_xlabel("Fraction of Features", fontsize=afs)
    ax[0,0].set_ylabel("Prop Hold Rate", fontsize=afs)
    ax[1,0].set_ylabel("Prop Hold Rate", fontsize=afs)
    
    if title is not None:
        fig.suptitle(title, fontsize=tfs, y=0.93)
    
    return fig

In [ ]:
q1t_vit16_rs = np.linspace(0.0, 4.5, 500)
q1t_vit16_plot_rs = q1t_vit16_rs / 64
q1e_vit16_rs = np.linspace(0.0, 31.9, 500)
q1e_vit16_plot_rs = q1e_vit16_rs / 64
_ = q1_plot_cert_vs_emp("vit16", q1t_vit16_rs, q1t_vit16_plot_rs, q1e_vit16_rs, q1e_vit16_plot_rs, title="Vision Transformer, SHAP top-25%")
plt.savefig("images/q1_emp_vit16.png", bbox_inches="tight")



q1t_resnet50_rs = q1t_vit16_rs
q1t_resnet50_plot_rs = q1t_vit16_plot_rs
q1e_resnet50_rs = q1e_vit16_rs
q1e_resnet50_plot_rs = q1e_vit16_plot_rs
_ = q1_plot_cert_vs_emp("resnet50", q1t_resnet50_rs, q1t_resnet50_plot_rs, q1e_resnet50_rs, q1e_resnet50_plot_rs, title="ResNet50, SHAP top-25%")
plt.savefig("images/q1_emp_resnet50.png", bbox_inches="tight")


q1t_roberta_rs = np.linspace(0.0, 0.41, 500)
q1t_roberta_plot_rs = np.linspace(0.0, 0.41, 500)
q1e_roberta_rs = q1t_roberta_rs
q1e_roberta_plot_rs = q1t_roberta_plot_rs
_ = q1_plot_cert_vs_emp("roberta", q1t_roberta_rs, q1t_roberta_plot_rs, q1e_roberta_rs, q1e_roberta_plot_rs, title="RoBERTa, SHAP top-25%")
plt.savefig("images/q1_emp_roberta.png", bbox_inches="tight")

In [ ]:

_ = q1_plot_cert_vs_emp("vit16", q1t_vit16_rs, q1t_vit16_plot_rs, q1e_vit16_rs, q1e_vit16_plot_rs,
                        title=None, show_lhs=True, subplot_titles=True)
plt.savefig("images/q1.png", bbox_inches="tight")

In [ ]:
#######################################################
# Certified accuracy drop plots
#######################################################

In [ ]:
def load_q2(model_type, lambd):
    if model_type == "roberta":
        csv_file = f"q2_{model_type}_q64_lam{lambd:.4f}.csv"
    else:
        csv_file = f"q2_{model_type}_psz28_q64_lam{lambd:.4f}.csv"
    csv_file = os.path.join(Q2_DIR, csv_file)
    return pd.read_csv(csv_file)

def q2_drops(df, rs, div_by_p=False):
    true_labels = df["true_label"].to_numpy()
    ones_labels = df["ones_label"].to_numpy()
    N = len(df)
    hit_bits = true_labels == ones_labels
    cert_rs = df["cert_r"].to_numpy()
    if div_by_p:
        ps = df["p"].to_numpy()
        cert_rs = cert_rs / ps
    drops = np.array([hit_bits[cert_rs >= r].sum() / N for r in rs])
    return drops


In [ ]:
q2_lambds = [16/16, 8/16, 4/16, 2/16, 1/16]

def lambd_str(lam):
    if abs(16 * lam - 1) < 1e-4:
        return "1/16"
    else:
        return f"{str(int(lambd*8))}/8"

q2_vit16_dfs = [(l,load_q2("vit16", l)) for l in q2_lambds]
q2_resnet50_dfs = [(l,load_q2("resnet50", l)) for l in q2_lambds]
q2_roberta_dfs = [(l,load_q2("roberta", l)) for l in q2_lambds]

In [ ]:
q2_rs = np.linspace(0.0, 10.01, 802)
q2_rs_roberta = np.linspace(0.0, 0.501, 502)

In [ ]:
### The version in the main paper

plt.clf()
lfs, afs, tfs = 14, 16, 18
fig, ax = plt.subplots(1,3, figsize=(6.4 * 2.7, 4.8 * 0.8))

for (lambd, df) in q2_vit16_dfs:
    drops = q2_drops(df, q2_rs)
    ax[0].plot(q2_rs/64, drops, label=f"λ={lambd_str(lambd)} ({drops[0]:.3f})")

for (lambd, df) in q2_resnet50_dfs:
    drops = q2_drops(df, q2_rs)
    ax[1].plot(q2_rs/64, drops, label=f"λ={lambd_str(lambd)} ({drops[0]:.3f})")

for (lambd, df) in q2_roberta_dfs:
    drops = q2_drops(df, q2_rs_roberta, div_by_p=True)
    ax[2].plot(q2_rs_roberta, drops, label=f"λ={lambd_str(lambd)} ({drops[0]:.3f})")


# ax[0].legend(loc="upper right", title="Vision Transformer", fontsize=lfs, title_fontsize=lfs)
# ax[1].legend(loc="upper right", title="ResNet50", fontsize=lfs, title_fontsize=lfs)
# ax[2].legend(loc="upper right", title="RoBERTa", fontsize=lfs, title_fontsize=lfs)

ax[0].legend(loc="upper right", fontsize=lfs, title_fontsize=lfs)
ax[1].legend(loc="upper right", fontsize=lfs, title_fontsize=lfs)
ax[2].legend(loc="upper right", fontsize=lfs, title_fontsize=lfs)

ax[0].set_title("Vision Transformer", fontsize=tfs)
ax[1].set_title("ResNet50", fontsize=tfs)
ax[2].set_title("RoBERTa", fontsize=tfs)

ax[0].set_ylabel("Certified Accuracy", fontsize=afs)
ax[0].tick_params(axis="y", labelsize=lfs)
for i in [0,1,2]:
    ax[i].set_yticks([0.0, 0.2, 0.4, 0.6, 0.8])
    ax[i].set_xlabel("Fraction of Features", fontsize=afs)
    ax[i].tick_params(axis="x", labelsize=lfs)
    ax[i].tick_params(axis="y", labelsize=lfs)

plt.savefig("images/q2.png", bbox_inches="tight")

In [ ]:
### The version in the appendix

plt.clf()

lfs, afs, tfs = 10, 14, 16
fig, ax = plt.subplots(3,1, figsize=(6.4 * 1, 4.8 * 2.7))

fig.subplots_adjust(hspace=0.4)

for (lambd, df) in q2_vit16_dfs:
    drops = q2_drops(df, q2_rs)
    ax[0].plot(q2_rs/64, drops, label=f"λ={lambd_str(lambd)} ({drops[0]:.3f})")

for (lambd, df) in q2_resnet50_dfs:
    drops = q2_drops(df, q2_rs)
    ax[1].plot(q2_rs/64, drops, label=f"λ={lambd_str(lambd)} ({drops[0]:.3f})")

for (lambd, df) in q2_roberta_dfs:
    drops = q2_drops(df, q2_rs_roberta, div_by_p=True)
    ax[2].plot(q2_rs_roberta, drops, label=f"λ={lambd_str(lambd)} ({drops[0]:.3f})")

ax[0].set_title("Vision Transformer", fontsize=tfs)
ax[1].set_title("ResNet50", fontsize=tfs)
ax[2].set_title("RoBERTa", fontsize=tfs)

ax[0].tick_params(axis="y", labelsize=lfs)
for i in [0,1,2]:
    ax[i].legend(loc="upper right", fontsize=lfs, title_fontsize=lfs)
    ax[i].set_ylabel("Certified Accuracy", fontsize=afs)
    ax[i].set_xlabel("Fraction of Features", fontsize=afs)
    ax[i].set_yticks([0.0, 0.2, 0.4, 0.6, 0.8])
    ax[i].tick_params(axis="x", labelsize=lfs)
    ax[i].tick_params(axis="y", labelsize=lfs)
#     ax[i].legend(loc="upper right")

plt.savefig("images/q2_appendix.png", bbox_inches="tight")

In [ ]:
########################################################

In [ ]:
def plot_workshop(model_type, trs, plot_trs,
                        title = None,
                        lfs = 12,
                        afs = 14,
                        tfs = 16):
    plt.clf()
    fig, ax = plt.subplots(1,3, figsize=(6.4 * 2.7, 4.8*0.7))

    lambds = [8/8, 4/8, 2/8, 1/8]
    q1t_dfs = [(l, load_q1_theory(model_type, "shap", 0.25, l)) for l in lambds]
    q2_dfs = [(l,load_q2(model_type, l)) for l in lambds]
    div_by_p = model_type.lower() == "roberta"
    
    for (lambd, df) in q1t_dfs:
        inc_drops = q1t_drops(df, trs, prop="inc", div_by_p=div_by_p)
        dec_drops = q1t_drops(df, trs, prop="dec", div_by_p=div_by_p)
        ax[0].plot(plot_trs, inc_drops, label=f"λ={int(lambd*8)}/8 ({inc_drops[0]:.3f})")
        ax[1].plot(plot_trs, dec_drops, label=f"λ={int(lambd*8)}/8 ({dec_drops[0]:.3f})")
            
    for (lambd, df) in q2_dfs:
        acc_drops = q2_drops(df, trs)
        ax[2].plot(plot_trs, acc_drops, label=f"λ={int(lambd*8)}/8 ({acc_drops[0]:.3f})")

#     fig.subplots_adjust(hspace=0.35)
        
    for i in range(3):
        ax[i].legend(loc="upper right", fontsize=lfs, title_fontsize=lfs)
        ax[i].set_yticks([0.0, 0.2, 0.4, 0.6, 0.8])
        ax[i].tick_params(axis="x", labelsize=lfs)
        ax[i].tick_params(axis="y", labelsize=lfs)

    ax[0].set_xlabel("Fraction of Features", fontsize=afs)
    ax[1].set_xlabel("Fraction of Features", fontsize=afs)
    ax[2].set_xlabel("Fraction of Features", fontsize=afs)
    
    ax[0].set_ylabel("Property Hold Rate", fontsize=afs)
    ax[1].set_ylabel("Property Hold Rate", fontsize=afs)
    ax[2].set_ylabel("Certified Accuracy", fontsize=afs)
    
    ax[0].set_title("Consistent and Incrementally Stable", fontsize=tfs)
    ax[1].set_title("Consistent and Decrementally Stable", fontsize=tfs)
    ax[2].set_title("Accuracy vs Decremental Stability", fontsize=tfs)
    
    if title is not None:
        fig.suptitle(title, fontsize=tfs, y=0.93)
    
    return fig

In [ ]:
q1t_vit16_rs = np.linspace(0.0, 4.5, 500)
q1t_vit16_plot_rs = q1t_vit16_rs / 64
q1e_vit16_rs = np.linspace(0.0, 31.9, 500)
q1e_vit16_plot_rs = q1e_vit16_rs / 64
_ = plot_workshop("vit16", q1t_vit16_rs, q1t_vit16_plot_rs, title=None)
plt.savefig("images/workshop_plot.png", bbox_inches="tight", transparent=True)